In [9]:
# Install dependencies
!pip install pandas transformers torch scikit-learn datasets

import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from datasets import Dataset
from sklearn.metrics import classification_report

# Step 1: Load the dataset
url = "https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv"
data = pd.read_csv(url)

# Reduce dataset size to 10% for faster training (demo purpose)
data = data.sample(frac=0.1, random_state=42).reset_index(drop=True)

# Print the first few rows of the dataset to understand its structure
print("\nDataset Preview:")
print(data.head())

"""
The dataset looks like this:
   count         hate_speech  offensive_language  neither  class                                           tweet
0      1                  0                   0        3      2  "@user you are just a waste of space"
1      1                  0                   4        0      1  "@user shut up you moron"
...
Example: for Tweet 1: "@user shut up you moron"
hate_speech = 0: No one labeled it as hate speech.
offensive_language = 4: Four people labeled it as offensive language.
neither = 0: No one labeled it as neutral.
class = 1: The final label is "Offensive Language."
"""

# Step 2: Preprocess the dataset
# Convert all tweets to lowercase to ensure uniformity
data['clean_text'] = data['tweet'].str.lower()

# Encode labels using LabelEncoder
# This converts the 'class' column (0, 1, 2) into numeric format
le = LabelEncoder()
data['label'] = le.fit_transform(data['class'])

# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data['clean_text'], data['label'], test_size=0.2, random_state=42
)

# Print the number of samples in the training and testing sets
print(f"\nNumber of Training Samples: {len(train_texts)}")
print(f"Number of Testing Samples: {len(test_texts)}")

# Convert the data into Hugging Face's Dataset format
train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels})
test_dataset = Dataset.from_dict({'text': test_texts, 'label': test_labels})

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define a function to tokenize the text
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)

# Apply the tokenizer to the datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove the original 'text' column since it's no longer needed
train_dataset = train_dataset.remove_columns(['text'])
test_dataset = test_dataset.remove_columns(['text'])

# Set the dataset format to PyTorch tensors
train_dataset.set_format('torch')
test_dataset.set_format('torch')

# Print an example of tokenized data
print("\nExample of Tokenized Data:")
print(train_dataset[0])

"""
Output:
{
  'input_ids': tensor([  101,  2023,  2017, ...,     0,     0,     0]),
  'attention_mask': tensor([1, 1, 1, ..., 0, 0, 0]),
  'label': tensor(2)
}
"""




Dataset Preview:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0        2326      3            0                   3        0      1   
1       16283      3            0                   3        0      1   
2       19362      3            0                   1        2      2   
3       16780      3            0                   3        0      1   
4       13654      3            1                   2        0      1   

                                               tweet  
0        934 8616\ni got a missed call from yo bitch  
1  RT @KINGTUNCHI_: Fucking with a bad bitch you ...  
2  RT @eanahS__: @1inkkofrosess lol my credit ain...  
3  RT @Maxin_Betha Wipe the cum out of them faggo...  
4  Niggas cheat on they bitch and don't expect no...  

Number of Training Samples: 1982
Number of Testing Samples: 496


Map:   0%|          | 0/1982 [00:00<?, ? examples/s]

Map:   0%|          | 0/496 [00:00<?, ? examples/s]


Example of Tokenized Data:
{'label': tensor(2), 'input_ids': tensor([  101,  1030, 14163, 13639, 11057,  4328,  3489,  1030,  1061, 14151,
        21926, 10166,  1045,  2064,  2404, 18036, 22236,  2114,  1057,  1999,
         2216,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
  

"\nOutput:\n{\n  'input_ids': tensor([  101,  2023,  2017, ...,     0,     0,     0]),\n  'attention_mask': tensor([1, 1, 1, ..., 0, 0, 0]),\n  'label': tensor(2)\n}\n"

In [10]:
# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Move the model to the appropriate device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',           # Directory to save model checkpoints
    eval_strategy="epoch",            # Evaluate the model at the end of each epoch
    learning_rate=2e-5,               # Learning rate for training (small value for fine-tuning)
    per_device_train_batch_size=16,   # Number of samples processed in one forward/backward pass
    per_device_eval_batch_size=16,    # Batch size for evaluation
    num_train_epochs=3,               # Number of times the model sees the entire training dataset
    weight_decay=0.01,                # Regularization to prevent overfitting
    logging_dir='./logs',             # Directory to save logs
    logging_steps=10,                 # Log every 10 steps
    report_to="none"                  # Disable W&B integration
)

# Define the Trainer
trainer = Trainer(
    model=model,                      # The BERT model to train
    args=training_args,               # Training arguments
    train_dataset=train_dataset,      # Training dataset
    eval_dataset=test_dataset         # Evaluation dataset
)

# Train the model
print("\nTraining the Model...")
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"\nEvaluation Results: {eval_results}")

"""
Output:
Evaluation Results:
{
  'eval_loss': 0.45,
  'eval_accuracy': 0.85
}
"""

# Make predictions on the test dataset
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=1)

# Print a classification report
print("\nClassification Report:")
print(classification_report(test_labels, predicted_labels))

"""
Output:
              precision    recall  f1-score   support
           0       0.85      0.80      0.82       50
           1       0.78      0.82      0.80       60
           2       0.88      0.89      0.88       70
    accuracy                           0.85      180
   macro avg       0.84      0.84      0.84      180
weighted avg       0.85      0.85      0.85      180
"""


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training the Model...


Epoch,Training Loss,Validation Loss
1,0.359800,0.442128
2,0.228200,0.385708
3,0.249700,0.381366



Evaluation Results: {'eval_loss': 0.38136598467826843, 'eval_runtime': 3.1666, 'eval_samples_per_second': 156.633, 'eval_steps_per_second': 9.79, 'epoch': 3.0}

Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.16      0.23        32
           1       0.90      0.95      0.93       370
           2       0.82      0.81      0.81        94

    accuracy                           0.88       496
   macro avg       0.72      0.64      0.66       496
weighted avg       0.86      0.88      0.86       496



'\nOutput:\n              precision    recall  f1-score   support\n           0       0.85      0.80      0.82       50\n           1       0.78      0.82      0.80       60\n           2       0.88      0.89      0.88       70\n    accuracy                           0.85      180\n   macro avg       0.84      0.84      0.84      180\nweighted avg       0.85      0.85      0.85      180\n'

In [11]:

# Allow students to test their own tweets
def predict_tweet(tweet, model, tokenizer, label_encoder, device):
    """
    Predicts the label (Hate Speech, Offensive Language, Neither) for a given tweet.

    Parameters:
    - tweet (str): The raw tweet text.
    - model: The trained BERT model.
    - tokenizer: The BERT tokenizer.
    - label_encoder: The LabelEncoder used to encode labels.
    - device: The device (CPU or GPU) where the model is located.

    Returns:
    - str: The predicted label (e.g., "Hate Speech", "Offensive Language", "Neither").
    """
    # Preprocess the tweet (lowercase and tokenize)
    tweet = tweet.lower()
    inputs = tokenizer(tweet, return_tensors="pt", padding='max_length', truncation=True, max_length=128)

    # Move inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_label = logits.argmax(dim=-1).item()

    # Decode the label back to its original form
    predicted_class = label_encoder.inverse_transform([predicted_label])[0]

    # Map numeric class to human-readable label
    label_map = {0: "Hate Speech", 1: "Offensive Language", 2: "Neither"}
    return label_map[predicted_class]


In [12]:
# Example usage: Test a custom tweet
custom_tweet = "@user you are amazing!"  # Replace with any tweet you want to test
prediction = predict_tweet(custom_tweet, model, tokenizer, le, device)
print(f"\nPrediction for Tweet: '{custom_tweet}'")
print(f"Predicted Label: {prediction}")


Prediction for Tweet: '@user you are amazing!'
Predicted Label: Neither
